## import nesessary Librarys

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re

## To read csv file and create dataframe

In [2]:
tweets = pd.read_csv('cleaned_fintech_data.csv')
tweets.head()

,Unnamed: 0,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp
0,0.0,Thu Jun 17 06:26:34 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Giving forth life is becoming a burden in Keny...,Giving forth life becoming burden Kenya This m...,"Sentiment(polarity=0.3194444444444445, subject...",0.3194444444444445,0.5305555555555556,en,0,...,reen_law,398,70,223,NaN,NaN,janetmachuka_,NaN,NaN,2021-06-17 06:26:34+00:00
1,1.0,Thu Jun 17 06:26:37 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Teenmaar - 26cr\nPanja - 32.5cr\nGabbarsingh -...,Teenmaar crPanja crGabbarsingh cr Khaleja Kuda...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,in,0,...,Amigo9999_,19047,132,1084,NaN,NaN,maheshblood,NaN,India,2021-06-17 06:26:37+00:00
2,2.0,Thu Jun 17 06:26:42 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Rei chintu 2013 lo Vachina Ad Nizam ne 2018 lo...,Rei chintu lo Vachina Ad Nizam ne lo kottaru f...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,hi,0,...,MallaSuhaas,47341,2696,2525,NaN,NaN,Hail_Kalyan,NaN,Vizag,2021-06-17 06:26:42+00:00
3,3.0,Thu Jun 17 06:26:44 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Today is World Day to Combat #Desertification ...,Today World Day Combat Restoring degraded land...,"Sentiment(polarity=0.25, subjectivity=0.65)",0.25,0.65,en,0,...,CIACOceania,7039,343,387,NaN,"Desertification, Drought, resilience",EdwardVrkic,NaN,Papua New Guinea,2021-06-17 06:26:44+00:00
4,4.0,Thu Jun 17 06:26:47 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Hearing #GregHunt say he's confident vaccines ...,Hearing say 's confident vaccines delivered li...,"Sentiment(polarity=0.5, subjectivity=0.8333333...",0.5,0.8333333333333334,en,0,...,MccarronWendy,26064,419,878,NaN,"GregHunt, Morrison",WriteWithDave,NaN,"Sydney, New South Wales",2021-06-17 06:26:47+00:00


## To get information abou the data

In [3]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5621 entries, 0 to 5620
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              5620 non-null   float64
 1   created_at              5621 non-null   object 
 2   source                  5621 non-null   object 
 3   original_text           5621 non-null   object 
 4   clean_text              5617 non-null   object 
 5   sentiment               5621 non-null   object 
 6   polarity                5621 non-null   object 
 7   subjectivity            5621 non-null   object 
 8   lang                    5621 non-null   object 
 9   favorite_count          5621 non-null   object 
 10  retweet_count           5621 non-null   object 
 11  original_author         5621 non-null   object 
 12  screen_count            5621 non-null   object 
 13  followers_count         5621 non-null   object 
 14  friends_count           5621 non-null   

In [5]:
tweets.dropna()

,Unnamed: 0,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp


In [6]:
len(tweets)

5621

In [7]:
tweets.head()

,Unnamed: 0,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp
0,0.0,Thu Jun 17 06:26:34 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Giving forth life is becoming a burden in Keny...,Giving forth life becoming burden Kenya This m...,"Sentiment(polarity=0.3194444444444445, subject...",0.3194444444444445,0.5305555555555556,en,0,...,reen_law,398,70,223,NaN,NaN,janetmachuka_,NaN,NaN,2021-06-17 06:26:34+00:00
1,1.0,Thu Jun 17 06:26:37 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Teenmaar - 26cr\nPanja - 32.5cr\nGabbarsingh -...,Teenmaar crPanja crGabbarsingh cr Khaleja Kuda...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,in,0,...,Amigo9999_,19047,132,1084,NaN,NaN,maheshblood,NaN,India,2021-06-17 06:26:37+00:00
2,2.0,Thu Jun 17 06:26:42 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Rei chintu 2013 lo Vachina Ad Nizam ne 2018 lo...,Rei chintu lo Vachina Ad Nizam ne lo kottaru f...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,hi,0,...,MallaSuhaas,47341,2696,2525,NaN,NaN,Hail_Kalyan,NaN,Vizag,2021-06-17 06:26:42+00:00
3,3.0,Thu Jun 17 06:26:44 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Today is World Day to Combat #Desertification ...,Today World Day Combat Restoring degraded land...,"Sentiment(polarity=0.25, subjectivity=0.65)",0.25,0.65,en,0,...,CIACOceania,7039,343,387,NaN,"Desertification, Drought, resilience",EdwardVrkic,NaN,Papua New Guinea,2021-06-17 06:26:44+00:00
4,4.0,Thu Jun 17 06:26:47 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Hearing #GregHunt say he's confident vaccines ...,Hearing say 's confident vaccines delivered li...,"Sentiment(polarity=0.5, subjectivity=0.8333333...",0.5,0.8333333333333334,en,0,...,MccarronWendy,26064,419,878,NaN,"GregHunt, Morrison",WriteWithDave,NaN,"Sydney, New South Wales",2021-06-17 06:26:47+00:00


## preprocessing Data

In [9]:
class PrepareData:
  def __init__(self,df):
    self.df=df
    
  def preprocess_data(self):
    tweets_df = self.df.loc[self.df['lang'] =="en"]

    
    #text Preprocessing
    tweets['clean_text']=tweets['clean_text'].astype(str)
    tweets['clean_text'] = tweets['clean_text'].apply(lambda x: x.lower())
    tweets['clean_text']= tweets['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
    
    #Converting tweets to list of words For feature engineering
    sentence_list = [tweet for tweet in tweets['clean_text']]
    word_list = [sent.split() for sent in sentence_list]
    # print(word_list)

    #Create dictionary which contains Id and word 
    word_to_id = corpora.Dictionary(word_list) #generate unique tokens
    #  we can see the word to unique integer mapping
    # print(word_to_id.token2id)
    # using bag of words(bow), we create a corpus that contains the word id and its frequency in each document.
    corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]
    # TFIDF

    return word_list, word_to_id, corpus_1


In [10]:
PrepareData_obj=PrepareData(tweets)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()

In [11]:
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)], [(26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)], [(45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1)], [(64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1)], [(67, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 2), (94, 2), (95, 1)], [(96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 1),

In [12]:
id_words = [[(id2word[id], count) for id, count in line] for line in corpus]

In [13]:
print(id_words)

[[('babies', 1), ('baby', 1), ('becoming', 1), ('birth', 1), ('bundles', 1), ('burden', 1), ('coz', 1), ('douglas', 1), ('expensiveturn', 1), ('formula', 1), ('forth', 1), ('gave', 1), ('giving', 2), ('handmpesa', 1), ('hard', 1), ('joy', 1), ('kenya', 1), ('life', 1), ('meeting', 1), ('mother', 1), ('needs', 1), ('nutritional', 1), ('nyaoko', 1), ('tears', 1), ('this', 1), ('time', 1)], [('appudu', 1), ('ayyindhi', 1), ('cr', 2), ('crbusinessman', 1), ('crgabbarsingh', 1), ('crpanja', 1), ('dookudu', 1), ('em', 1), ('endhuku', 1), ('ginjukuntaru', 1), ('khaleja', 1), ('kottadaniki', 1), ('kuda', 1), ('mari', 1), ('ra', 1), ('teenmaar', 1), ('undhi', 1), ('urike', 1), ('varaku', 1)], [('ad', 1), ('agv', 1), ('chintu', 1), ('day1', 1), ('disco', 1), ('entra', 1), ('esedhi', 1), ('films', 1), ('inka', 1), ('kottaledhu', 1), ('kottaru', 1), ('lo', 2), ('matho', 1), ('meru', 1), ('ne', 1), ('nizam', 1), ('pattinndhi', 1), ('rei', 1), ('vachina', 1)], [('all', 1), ('away', 1), ('brings', 1)

## Build LDA model

In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
pprint(lda_model.print_topics())

[(0,
  '0.024*"carbon" + 0.010*"trend" + 0.008*"year" + 0.008*"emissions" + '
  '0.008*"climate" + 0.007*"bday" + 0.007*"character" + 0.006*"away" + '
  '0.006*"the" + 0.006*"like"'),
 (1,
  '0.034*"carbon" + 0.029*"oil" + 0.029*"emissions" + 0.019*"s" + '
  '0.018*"neutral" + 0.018*"amp" + 0.016*"government" + 0.015*"scope" + '
  '0.014*"the" + 0.013*"i"'),
 (2,
  '0.024*"tax" + 0.024*"protecting" + 0.024*"rights" + 0.020*"kuda" + '
  '0.014*"coal" + 0.013*"covid" + 0.012*"carbon" + 0.012*"resource" + '
  '0.012*"new" + 0.012*"reform"'),
 (3,
  '0.035*"kuda" + 0.021*"lo" + 0.015*"rc" + 0.015*"ni" + 0.011*"gt" + '
  '0.011*"aa" + 0.010*"ani" + 0.010*"ki" + 0.009*"pk" + 0.009*"malli"'),
 (4,
  '0.018*"carbon" + 0.014*"amp" + 0.010*"money" + 0.008*"kuda" + '
  '0.007*"mobile" + 0.007*"pm" + 0.007*"using" + 0.005*"peat" + '
  '0.005*"airtime" + 0.005*"grouse"')]


In [16]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('carbon', 0.024218192),
   ('trend', 0.010427372),
   ('year', 0.007757467),
   ('emissions', 0.0076717306),
   ('climate', 0.0075800307),
   ('bday', 0.007048122),
   ('character', 0.00700511),
   ('away', 0.006198688),
   ('the', 0.0061525367),
   ('like', 0.0058576562)]),
 (1,
  [('carbon', 0.03415686),
   ('oil', 0.029223524),
   ('emissions', 0.028720446),
   ('s', 0.019260619),
   ('neutral', 0.017793769),
   ('amp', 0.017781623),
   ('government', 0.015660478),
   ('scope', 0.01549459),
   ('the', 0.013858135),
   ('i', 0.013467368)]),
 (2,
  [('tax', 0.023772417),
   ('protecting', 0.023677329),
   ('rights', 0.023616008),
   ('kuda', 0.019635295),
   ('coal', 0.013615177),
   ('covid', 0.012865034),
   ('carbon', 0.012179792),
   ('resource', 0.011967632),
   ('new', 0.011917922),
   ('reform', 0.011909954)]),
 (3,
  [('kuda', 0.034580972),
   ('lo', 0.020984126),
   ('rc', 0.015490771),
   ('ni', 0.014822623),
   ('gt', 0.011429276),
   ('aa', 0.010702126),
   ('ani'

##  Compute Perplexity

In [17]:
#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -7.024655816092819

 Ldamodel Coherence Score/Accuracy on Tweets:  0.5104868990255852


## Visualize the topics

In [19]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.212539 -0.295377       1        1  25.923394
0     -0.158967  0.102845       2        1  24.236652
1     -0.184646 -0.047311       3        1  18.032684
2      0.262987  0.258371       4        1  17.883415
4     -0.131913 -0.018528       5        1  13.923855, topic_info=            Term         Freq        Total Category  logprob  loglift
338          oil   738.000000   738.000000  Default  30.0000  30.0000
88     emissions   985.000000   985.000000  Default  29.0000  29.0000
1603  protecting   593.000000   593.000000  Default  28.0000  28.0000
1608      rights   591.000000   591.000000  Default  27.0000  27.0000
38          kuda  1967.000000  1967.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
112          via    67.679379    79.687688   Topic5  -5.6627   1.8082
166          one    98.866250   226.260338   Topic5  -5.2837   1.1436
593       people    80.698379   156.370044   Topic5  -5.4867   1.3101
38          kuda   151.477911  1967.404654   Topic5  -4.8570  -0.5925
93             s    72.772155   822.353172   Topic5  -5.5901  -0.4533

[238 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3598      3  0.998186        8
615       1  0.997330       aa
96        2  0.766035  account
96        3  0.231494  account
1588      4  0.998666  against
...     ...       ...      ...
479       2  0.996186    would
2587      1  0.993861       ye
456       2  0.996630     year
1587      5  0.991274      you
614       3  0.997182     zero

[263 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 5])